In [ ]:
__author__ = "Anirban Dey, MSc"
__license__ = "GNU General Public License v3.0"
__version__ = "1.0"
__maintainer__ = "A. Dey"
__status__ = "Pre-Production"

import pandas as pd
import numpy as np
import time
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import binarize
import time
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import warnings
import pareto
warnings.filterwarnings('ignore')
from numpy.linalg import norm as l2
import MOO_Libs as MOO_libs

In [ ]:
path = "D:/Data/Arsenic Data/Complete_data_Differential_FC2.0.xlsx"
df1 = pd.read_excel(path,sheet_name=0)
pvalId = ['Identifier','p Value_With_Lesion_Vs_Normal','p Value_Without_Lesion_Vs_Normal']
foldchange = ['FC (abs)_Without_Lesion_Vs_Normal','FC (abs)_With_Lesion_Vs_Normal']

Normal_s0 = ['Normal1_Fold_Exp', 'Normal10_Fold_Exp',
       'Normal11_Fold_Exp', 'Normal12_Fold_Exp', 'Normal13_Fold_Exp',
       'Normal14_Fold_Exp', 'Normal15_Fold_Exp', 'Normal16_Fold_Exp',
       'Normal17_Fold_Exp', 'Normal2_Fold_Exp', 'Normal3_Fold_Exp',
       'Normal4_Fold_Exp', 'Normal5_Fold_Exp', 'normal6_Fold_Exp',
       'Normal7_Fold_Exp', 'Normal8_Fold_Exp', 'Normal9_Fold_Exp']
#17
withoutles_s0 = [ 'Woutles1_Fold_Exp',
       'Woutles10_Fold_Exp', 'Woutles11_Fold_Exp', 'Woutles12_Fold_Exp',
       'Woutles13_Fold_Exp', 'Woutles14_Fold_Exp', 'Woutles15_Fold_Exp',
       'Woutles16_Fold_Exp', 'Woutles17_Fold_Exp', 'Woutles18_Fold_Exp',
       'Woutles19_Fold_Exp', 'Woutles2_Fold_Exp', 'Woutles20_Fold_Exp',
       'Woutles3_Fold_Exp', 'Woutles4_Fold_Exp', 'Woutles5_Fold_Exp',
       'Woutles6_Fold_Exp', 'Woutles7_Fold_Exp', 'Woutles8_Fold_Exp',
       'Woutles9_Fold_Exp']
#20
withles_s0 = ['Wles1_Fold_Exp', 'Wles10_Fold_Exp', 'Wles11_Fold_Exp',
       'Wles12_Fold_Exp', 'Wles13_Fold_Exp', 'Wles14_Fold_Exp',
       'Wles15_Fold_Exp', 'Wles16_Fold_Exp', 'Wles17_Fold_Exp',
       'Wles18_Fold_Exp', 'Wles19_Fold_Exp', 'Wles2_Fold_Exp',
       'Wles20_Fold_Exp', 'Wles21_Fold_Exp', 'Wles22_Fold_Exp',
       'Wles3_Fold_Exp', 'Wles4_Fold_Exp', 'Wles5_Fold_Exp', 'Wles6_Fold_Exp',
       'Wles7_Fold_Exp', 'Wles9_Fold_Exp']
#21
17+20+21


In [ ]:
df1_pval = df1[pvalId]
df1_foldchng = df1[foldchange]

df1_Norm = df1[Normal_s0]
df1_withoutles = df1[withoutles_s0]
df1_withles = df1[withles_s0]

df1_Norm = df1_Norm.T
df1_withles = df1_withles.T
df1_withoutles = df1_withoutles.T

FC_wol = df1_foldchng['FC (abs)_Without_Lesion_Vs_Normal']
FC_wl = df1_foldchng['FC (abs)_With_Lesion_Vs_Normal']

"""
df1_Norm['Label'] = pd.Series(0, index=df1_Norm.index)
df1_withles['Label'] = pd.Series(1,index=df1_withles.index)
df1_withoutles['Label'] = pd.Series(1,index=df1_withoutles.index)
"""
frames = [df1_Norm, df1_withles, df1_withoutles]
#frames = [df1_withles, df1_withoutles]
#frames = [df1_Norm, df1_withoutles]
#frames = [df1_Norm, df1_withles]
result = pd.concat(frames)

#P-value constrain vector creation 
'''
pval_wl = df1_pval['p Value_With_Lesion_Vs_Normal'].head(1802)
pval_wol = df1_pval['p Value_Without_Lesion_Vs_Normal'].head(1802)'''
pval_wl = df1_pval['p Value_With_Lesion_Vs_Normal']
pval_wol = df1_pval['p Value_Without_Lesion_Vs_Normal']
pval_wl = pval_wl.values #numpy array conversion
pval_wol = pval_wol.values #numpy array conversion

FC_wol = FC_wol.values #numpy array conversion
FC_wl = FC_wl.values #numpy array conversion'''

In [ ]:
#Input and class seperation

X = result.drop('Label', axis=1)
y = result[['Label']]





#X = X.reset_index(drop = True)
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)
X_std = pd.DataFrame(X_std)
from scipy.stats import zscore
Train_data_zScore = X_std.apply(zscore)

In [ ]:
#GENE selection Using PSO
DMTN = 19953
Exploit = 1
Explore = 0
i = 50
j = 19953
NOP = 50
pmin = np.zeros(DMTN)
pmax = np.ones(DMTN)
position = np.random.rand(50,DMTN)
velocity = np.random.rand(50,DMTN)
#vmax = np.empty(DMTN); vmax.fill(0.10)
#vmin = np.empty(DMTN); vmax.fill(-0.10)
K = 4 

# Initialize the pbest
PBest = []
GL2 = []
funfit = []
for i in range(NOP):
    GL2.append([0,0])
    PBest.append([0,19953,1,0])
    funfit.append(0)

GBest = [0,19950,1,0]

In [ ]:
print("__Enter Log File Name__")
file = input()
#Generate Log File
Mode = 'w'
f = open(file,Mode)
thresh = 0.7 #Initialization 
Caption = "Iteration,G-best Accuracy,Number of Gene,f1-score,P-Value,Norm_FC(Abs)\n"
f.write(Caption)
print("Log Entry started...")


th_position = np.ones((50, 19950))
P_Best_Archive = []
#f.close()
flag = 0

In [ ]:
time_start = time.clock()
#Loop for iteration 
NORUN = 350
for itr in range(NORUN):
    #for a single Iteration
    CostForItr = [] # Tracks cost of every particle for a particular iteration
    
    th_position = binarize(position[:,:19950], threshold = thresh, copy=True)
    cost = []
    for i in range(NOP):
        #print("Updation for particle-",i)
        Pos = th_position[i,:] #for a single Particle

        #get the position of 1's in each of the gene exp. of a particle
        
        POS_INDEX = np.where(Pos==1)
        POS_INDEX = POS_INDEX[0]
        POS_INDEX = np.ndarray.tolist(POS_INDEX)
    
    
        #....... FITNESS FUNCTION ............
        
        cost.append(MOO_libs.fitness(X,y,POS_INDEX,[position[i,19950], position[i,19951], position[i,19952]],df1_pval,df1_foldchng))
        
    radius = MOO_libs.comp_rad()
    
    present_lbest = MOO_libs.lbest_particles(cost,radius)
    
   
    #select pbest 
    for i,j,k in zip(present_lbest,np.array(PBest),range(NOP)):
        if i.mean() < j.mean():
            PBest[k] = i
        
        
    
    GBest = pareto.eps_sort(PBest)[L2_Dist(pareto.eps_sort(PBest))] #indexing the min l2-distance of esp sort
    
    
    #Updatiton Process
    ### This code belongs to Anirban Dey Research Scholar @ Aphy_Caluniv ###
    
    #Calculation of Particle Velocity and Position
    c1=2; c2=2; wstart=0.9; wend=0.4 
    inertia = ((NORUN - itr)*(wstart - wend))/NORUN + wend

    for i in range(NOP):
        for j in range(DMTN):
            r1 = np.random.uniform(0,1)
            r2 = np.random.uniform(0,1)
            #Particle - Velocity update
            pB_Sum = 0 
            gB_Sum = 0
            pB_Sum += sum(PBest[i][:])/4.0
            gB_Sum += sum(GBest)/4.0
            
            velocity[i][j] = inertia*float(velocity[i][j])+c1*r1*(pB_Sum-float(position[i][j]))+c2*r2*(gB_Sum-float(position[i][j]))
            #particle - Position update
            position[i][j] = position[i][j] + velocity[i][j]
    #run your code
    #strin = str(itr)+","+str(Gbest_Accuracy*100)+","+str(len(redu_fvect))+","+str(f1Score)+","+str(max(CLASSIFICATION_ACCURACY))+","+str(redu_fvect)+"\n"
    strin = str(itr)+","+str(GBest[0])+","+str(GBest[1])+","+str(GBest[2])+","+str(GBest[3])+"\n"
    f.write(strin)
    time_elapsed = (time.clock() - time_start)
    Msg = "Iteration: %d Accuracy=%3.2f Gene=%d Computational Time:%f" % (itr+1,GBest[0]*100,GBest[1],time_elapsed)
    print(Msg)
    

time_elapsed = (time.clock() - time_start)
print("Computational Time:",time_elapsed)
f.close()